# Benchmark

This notebook provides a straightforward way to compare the PyZX optimization routines against other approaches on a standard set of benchmark circuits.

First we execute the standard set of imports:

In [1]:
import random, math, os, time
import sys; sys.path.append('..')
import pyzx as zx
%config InlineBackend.figure_format = 'svg'

The following class is some boilerplate around the simplification routines so that we can more easily print the relevant metrics

In [2]:
class CircuitComparer:
    def __init__(self, dirname, before, after):
        self.fname_before = os.path.join(dirname, before)
        if after:
            self.fname_after = os.path.join(dirname, after)
        else:
            self.fname_after = ""
        self.fname_tpar = ""
        if before.find('before') != -1:
            self.name = before[:-7]
        else:
            self.name = before
        self.has_run = False
    def __str__(self):
        return "CircuitComparer({}, {})".format(self.name, str(self.has_run))
    def __repr__(self):
        return str(self)
    
    def run(self):
        if self.has_run: return True
        c = zx.Circuit.load(self.fname_before).to_basic_gates()
        if self.fname_after:
            c_opt = zx.Circuit.load(self.fname_after).to_basic_gates()
            self.t_opt = c_opt.tcount()
        else:
            self.t_opt = '-'
        self.qubits = c.qubits
        if self.fname_tpar:
            c2 = zx.Circuit.load(self.fname_tpar)
            self.tpar = c2.tcount()
        else: self.tpar = "-"
        self.gatecount = len(c.gates)
        self.t_before = c.tcount()
        g = c.to_graph()
        t = time.time()
        g = zx.simplify.teleport_reduce(g)
        self.t_after = zx.tcount(g)
        self.time_simpl = time.time() - t
        t = time.time()
        c_opt = zx.extract.streaming_extract(g).to_basic_gates()
        c_opt = zx.optimize.basic_optimization(c_opt).to_basic_gates()
        self.c_opt = c_opt
        c_id = c.adjoint()
        c_id.add_circuit(c_opt)
        g = c_id.to_graph()
        zx.simplify.full_reduce(g)
        if g.num_vertices() == 2*len(g.inputs):
            self.verified = "Y"
        else: self.verified = "-"
        
        self.extracts = True
        self.time_extr = 0.0
#         try: 
#             c2 = zx.extract.streaming_extract(g,quiet=True)
#             self.time_extr = time.time() - t
#         except Exception:
#             self.extracts = False
#             self.time_extr = -1
        self.has_run = True
        
        return True
    
    def get_output(self):
        if not self.has_run:
            self.run()
        s = self.name.ljust(20) + str(self.qubits).rjust(7)
        s += str(self.gatecount).rjust(8) + str(self.t_before).rjust(9) + str(self.t_opt).rjust(7) 
        s += str(self.tpar).rjust(8) + str(self.t_after).rjust(8)
        s += "{:.2f}".format(self.time_simpl).rjust(11)
        #s += "{:.2f}".format(self.time_extr).rjust(12)
        s += "       " + self.verified
        #s += str(self.hcount).rjust(8) + str(self.vcount).rjust(8)
        return s

Next we define a function that loads in a directory of circuit files. Note that the directory we target has up to 3 versions of each circuit:

* circuit_before   - This is the original circuit with any modifications, taken from the [Github page](https://github.com/njross/optimizer) of [[1]](#NRSCM)
* circuit_after    - This is the circuit produced by the optimization routines of [[1]](#NRSCM).
* circuit_tpar.qc  - This is the circuit produced by the Tpar algorithm [[2]](#Tpar).
  
<a id="NRSCM"></a>
[1] [Nam, Ross, Su, Childs, Maslov - Automated optimization of large quantum circuits with continuous parameters](https://www.nature.com/articles/s41534-018-0072-4)

<a id="Tpar"></a>
[2] [Amy, Maslov, Mosca - Polynomial-time T-depth Optimization of Clifford+T circuits via Matroid Partitioning](https://arxiv.org/abs/1303.2042)

In [3]:
def load_circuits(directory):
    d = directory
    beforefiles = []
    afterfiles = []
    tparfiles = []
    for f in os.listdir(d):
        if not os.path.isfile(os.path.join(d,f)): continue
        if f.find('before') != -1:
            beforefiles.append((f,d))
        elif f.find('tpar') != -1:
            tparfiles.append((f,d))
        elif f.find('.qc') != -1 or f.find('.tfc') != -1:
            beforefiles.append((f,d))
        else: afterfiles.append((f,d))
    
    circuits = []
    for f, d in beforefiles:
        if f.find('before') == -1:
            n = os.path.splitext(f)[0]
        else: n = f[:-7]
        for f2,d2 in afterfiles:
            if d!=d2: continue
            if f2.startswith(n):
                c = CircuitComparer(d, f, f2)
                circuits.append(c)
                break
        else:
            c = CircuitComparer(d, f, '')
            circuits.append(c)
        for f2,d2 in tparfiles:
            if d!=d2: continue
            if f2.startswith(n):
                circuits[-1].fname_tpar = os.path.join(d2,f2)
    
    return circuits

dir_fast_circuits = os.path.join('..', 'circuits', 'Fast')
fast_circuits = load_circuits(dir_fast_circuits)

The directory we target contains a subset of all benchmark circuits, chosen for given quick results. The following cell giving benchmark results of these circuits should therefore only take a few seconds to run. For the benchmarks of slower circuits see [below](#slowbench).
The columns have the following meaning:

* `Circuit     ` - The name of the circuit
* `qubits      ` - Amount of qubits in the circuit
* `G-count     ` - Gate count of original circuit
* `T-before    ` - Amount of T-gates in original circuit
* `T-NRSCM     ` - Amount of T-gates in optimised circuit of [[1]](#NRSCM)
* `T-par       ` - Amount of T-gates in optimised circuit of [[2]](#Tpar)
* `T-PyZX      ` - Amount of T-gates in optimised circuit made by PyZX
* `Time-Simp   ` - The time taken for running the simplification routine on the circuit
* `Time-Extract` - The time taken for extracting the circuit after the simplification

Note that not all circuits were present in the papers [[1]](#NRSCM) and [[2]](#Tpar) in which case the relevant columns are empty.

In [4]:
print("Circuit".ljust(20), "qubits", "G-count", "T-before", "T-NRSCM", " T-par", " T-PyZX", " Time-Simp","Verified")
for c in fast_circuits:
    print(c.get_output())

Circuit              qubits G-count T-before T-NRSCM  T-par  T-PyZX  Time-Simp Verified
Adder8                   23     637      266     56       -      56       0.30       Y
adder_8                  24    1014      399    215     215     173       0.67       Y
barenco_tof_10           19     514      224    100     100     100       0.18       Y
barenco_tof_3             5      66       28     16      16      16       0.01       Y
barenco_tof_4             7     130       56     28      28      28       0.04       Y
barenco_tof_5             9     194       84     40      40      40       0.06       Y
csla_mux_3_original      15     190       70     64       -      62       0.05       Y
csum_mux_9_corrected     30     448      196     84       -      84       0.12       Y
gf2^10_mult              30    1709      700    410     410     410       0.67       Y
gf2^4_mult               12     275      112     68      68      68       0.09       Y
gf2^5_mult               15     429      1

<a id="slowbench"></a>
And now we do the benchmark on the slower circuits. Note that this can take up to half an hour to complete.

In [6]:
dir_slow_circuits = os.path.join('..', 'circuits', 'Slow')
slow_circuits = load_circuits(dir_slow_circuits)
print("Circuit".ljust(20), "qubits", "G-count", "T-before", "T-NRSCM", " T-par", " T-PyZX", " Time-Simp", " Time-Extract")
for c in slow_circuits:
    print(c.get_output())

Circuit              qubits G-count T-before T-NRSCM  T-par  T-PyZX  Time-Simp  Time-Extract
Adder16                  47    1437      602    120       -     120       0.58       14.36
Adder32                  95    3037     1274    248       -     248       1.59      128.39
Adder64                 191    6237     2618    504       -     504       3.22      510.34
gf2^16_mult              48    4397     1792   1040    1040    1040       4.65        0.16
ham15-high.qc            20    6010     2457      -       -    1019       7.39       22.70
ham15-med.qc             17    1436      574      -       -     212       1.28        1.83
mod_adder_1024           28    4855     1995   1011    1011    1011       2.31        8.45
QFT32                    32    1562      918    368       -     368       0.24        0.04
QFTAdd16                 32    1822     1026    402       -     402       0.46        0.94
QFTAdd32                 64    4814     2754   1042       -    1042       1.34       35.

In [5]:
def load_pf_circuits(directory):
    d = directory
    circuits = []
    for f in os.listdir(d):
        if not os.path.isfile(os.path.join(d,f)): continue
        if f.find('after') == -1: continue
        c = CircuitComparer(d,f,f)
        circuits.append(c)
    return circuits

In [6]:
dir_pf_circuits = os.path.join('..', 'circuits', 'PF')
pf_circuits = load_pf_circuits(dir_pf_circuits)
print("Circuit".ljust(20), "qubits", "G-count", "T-before", "T-NRSCM", " T-par", " T-PyZX", " Time-Simp", " Time-Extract")
for c in pf_circuits:
    print(c.get_output())

Circuit              qubits G-count T-before T-NRSCM  T-par  T-PyZX  Time-Simp  Time-Extract
pf1_100_after           100    1500      400    400       -     400      50.88        9.86
pf1_10_after             10     150       40     40       -      40       0.05        0.02
pf1_20_after             20     300       80     80       -      80       0.11        0.11
pf1_30_after             30     450      120    120       -     120       0.66        0.27
pf1_40_after             40     600      160    160       -     160       0.48        0.44
pf1_50_after             50     750      200    200       -     200       8.99        5.19
pf1_60_after             60     900      240    240       -     240       1.61        1.36
pf1_70_after             70    1050      280    280       -     280       4.58        1.91
pf1_80_after             80    1200      320    320       -     320      42.77        8.02
pf1_90_after             90    1350      360    360       -     360      93.12       30.

KeyboardInterrupt: 